In [13]:
import pandas as pd

In [14]:
benchmark_scores = pd.read_csv('../data/processed/benchmark_scores_2021_2023_cleaned.csv')
subject_groups = pd.read_csv('../data/processed/subject_groups.csv')

In [15]:
benchmark_scores.head()

,year_,university_code,major_code,major_name,subject_group_code,admission_method,score_scale,benchmark_score
0,2021,FTU2,NTS02,"Kế toán, Tài chính - Ngân hàng",D07,Tốt nghiệp THPT,30,28.4
1,2021,GDU,7220201,Ngôn ngữ Anh,D01,Tốt nghiệp THPT,30,16.5
2,2021,GDU,7220201,Ngôn ngữ Anh,D15,Tốt nghiệp THPT,30,16.5
3,2021,GDU,7220201,Ngôn ngữ Anh,XDHB,Xét học bạ,30,16.5
4,2021,GDU,7310608,Đông phương học,D01,Tốt nghiệp THPT,30,16.5


In [16]:
subject_groups.head()

,subject_group_code,subject_1,subject_2,subject_3
0,A00,Toán,Vật lí,Hóa học
1,A01,Toán,Vật lí,Tiếng Anh
2,B00,Toán,Hóa học,Sinh học
3,C00,Ngữ văn,Lịch sử,Địa lí
4,D01,Ngữ văn,Toán,Tiếng Anh


In [17]:
df = pd.merge(benchmark_scores,subject_groups,on='subject_group_code',how='left')

In [18]:
df.head()

,year_,university_code,major_code,major_name,subject_group_code,admission_method,score_scale,benchmark_score,subject_1,subject_2,subject_3
0,2021,FTU2,NTS02,"Kế toán, Tài chính - Ngân hàng",D07,Tốt nghiệp THPT,30,28.4,Toán,Hóa học,Tiếng Anh
1,2021,GDU,7220201,Ngôn ngữ Anh,D01,Tốt nghiệp THPT,30,16.5,Ngữ văn,Toán,Tiếng Anh
2,2021,GDU,7220201,Ngôn ngữ Anh,D15,Tốt nghiệp THPT,30,16.5,Ngữ văn,Địa lí,Tiếng Anh
3,2021,GDU,7220201,Ngôn ngữ Anh,XDHB,Xét học bạ,30,16.5,NaN,NaN,NaN
4,2021,GDU,7310608,Đông phương học,D01,Tốt nghiệp THPT,30,16.5,Ngữ văn,Toán,Tiếng Anh


In [19]:
df.shape

(4252, 11)

In [21]:
df = df[['year_','university_code','major_code','major_name','subject_group_code','subject_1','subject_2','subject_3',\
         'admission_method','score_scale','benchmark_score']]
df.shape

(4252, 11)

In [22]:
df.head()

,year_,university_code,major_code,major_name,subject_group_code,subject_1,subject_2,subject_3,admission_method,score_scale,benchmark_score
0,2021,FTU2,NTS02,"Kế toán, Tài chính - Ngân hàng",D07,Toán,Hóa học,Tiếng Anh,Tốt nghiệp THPT,30,28.4
1,2021,GDU,7220201,Ngôn ngữ Anh,D01,Ngữ văn,Toán,Tiếng Anh,Tốt nghiệp THPT,30,16.5
2,2021,GDU,7220201,Ngôn ngữ Anh,D15,Ngữ văn,Địa lí,Tiếng Anh,Tốt nghiệp THPT,30,16.5
3,2021,GDU,7220201,Ngôn ngữ Anh,XDHB,NaN,NaN,NaN,Xét học bạ,30,16.5
4,2021,GDU,7310608,Đông phương học,D01,Ngữ văn,Toán,Tiếng Anh,Tốt nghiệp THPT,30,16.5


In [29]:
majors = df.loc[:,['major_code','major_name']]

In [40]:
majors['major_name'].value_counts().to_frame()

,count
major_name,
Quản trị kinh doanh,194
Luật,177
Công nghệ thông tin,147
Kế toán,139
Ngôn ngữ Anh,138
...,...
Kỹ thuật công trình xây dựng,1
Quản trị nhà hàng khách sạn,1
Tài chính và kiểm soát,1


In [41]:
majors.loc[majors['major_name'].str.contains('kinh doanh'),:].value_counts().to_frame()

,,count
major_code,major_name,
7340101,Quản trị kinh doanh,109
7340101C,Quản trị kinh doanh,20
7340129,Quản trị kinh doanh,11
7340101_WE4,Quản trị kinh doanh,4
7340101_AND,Quản trị kinh doanh,3
7340101_UN,Quản trị kinh doanh,3
7340002,Quản trị kinh doanh,3
7340101K,Quản trị kinh doanh,3
7340101_TH,Quản trị kinh doanh,3


Các đặt trưng:   
+ `year_`
+ university_code
+ major_code
+ major_name